# Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.linear_model import Ridge
from matplotlib.colors import Normalize
import networkx as nx
import itertools
from tqdm import tqdm
from scipy.signal import welch
import matplotlib.cm as cm
import seaborn as sns
from collections import defaultdict
from sklearn.decomposition import PCA
import json
from numpy.linalg import eigvals
import os
import scipy.io as sio

# Utils

In [2]:
def scale_spectral_radius(W, target_radius=0.95):
    """
    Scales a matrix W so that its largest eigenvalue magnitude = target_radius.
    """
    eigvals = np.linalg.eigvals(W)
    radius = np.max(np.abs(eigvals))
    if radius == 0:
        return W
    return (W / radius) * target_radius

def augment_state_with_squares(x):
    """
    Given state vector x in R^N, return [ x, x^2, 1 ] in R^(2N+1).
    We'll use this for both training and prediction.
    """
    x_sq = x**2
    return np.concatenate([x, x_sq, [1.0]])  # shape: 2N+1

# Baseline ESN

In [3]:
class ESN3D:
    """
    Dense random ESN for 3D->3D single-step.
    Teacher forcing for training, autoregressive for testing.
    """
    def __init__(self,
                 reservoir_size=300,
                 input_dim=62,
                 spectral_radius=0.95,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=42):
        self.reservoir_size = reservoir_size
        self.spectral_radius = spectral_radius
        self.input_scale = input_scale
        self.leaking_rate = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed = seed

        np.random.seed(self.seed)
        W = np.random.randn(reservoir_size, reservoir_size)*0.1
        W = scale_spectral_radius(W, self.spectral_radius)
        self.W = W

        np.random.seed(self.seed+1)
        self.W_in = (np.random.rand(reservoir_size,input_dim) - 0.5)*2.0*self.input_scale
        # self.W_in = np.random.uniform(-self.input_scale, self.input_scale, (reservoir_size, 3))

        self.W_out = None
        self.x = np.zeros(reservoir_size)

    def reset_state(self):
        self.x = np.zeros(self.reservoir_size)

    def _update(self, u):
        pre_activation = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre_activation)
        alpha = self.leaking_rate
        self.x = (1.0 - alpha)*self.x + alpha*x_new

    def collect_states(self, inputs, discard=100):
        self.reset_state()
        states = []
        for val in inputs:
            self._update(val)
            states.append(self.x.copy())
        states = np.array(states)
        return states[discard:], states[:discard]

    def fit_readout(self, train_input, train_target, discard=100):
        states_use, _ = self.collect_states(train_input, discard=discard)
        targets_use = train_target[discard:]
        # X_aug = np.hstack([states_use, np.ones((states_use.shape[0],1))])

        # polynomial readout
        X_list = []
        for s in states_use:
            X_list.append(augment_state_with_squares(s))
        X_aug = np.array(X_list)  # shape => [T-discard, 2N+1]

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_

    def predict_autoregressive(self, initial_input, n_steps):
        preds = []
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            # x_aug = np.concatenate([self.x, [1.0]])
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
            current_in = out
        return np.array(preds)
    
    def predict_open_loop(self, test_input):
        preds = []
        for true_input in test_input:
            self._update(true_input)
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)


# SCR

In [4]:
class CR3D:
    """
    Cycle (ring) reservoir for 3D->3D single-step,
    teacher forcing for training, autoregressive for testing.
    """
    def __init__(self,
                 reservoir_size=300,
                 input_dim=62,
                 spectral_radius=0.95,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=42):
        self.reservoir_size = reservoir_size
        self.spectral_radius = spectral_radius
        self.input_scale = input_scale
        self.leaking_rate = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed = seed

        np.random.seed(self.seed)
        W = np.zeros((reservoir_size, reservoir_size))
        for i in range(reservoir_size):
            j = (i+1) % reservoir_size
            W[i, j] = 1.0
        W = scale_spectral_radius(W, self.spectral_radius)
        self.W = W
        
        np.random.seed(self.seed+1)
        self.W_in = (np.random.rand(reservoir_size,input_dim) - 0.5)*2.0*self.input_scale

        self.W_out = None
        self.x = np.zeros(reservoir_size)

    def reset_state(self):
        self.x = np.zeros(self.reservoir_size)

    def _update(self, u):
        pre_activation = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre_activation)
        alpha = self.leaking_rate
        self.x = (1.0 - alpha)*self.x + alpha*x_new

    def collect_states(self, inputs, discard=100):
        self.reset_state()
        states = []
        for val in inputs:
            self._update(val)
            states.append(self.x.copy())
        states = np.array(states)
        return states[discard:], states[:discard]

    def fit_readout(self, train_input, train_target, discard=100):
        states_use, _ = self.collect_states(train_input, discard=discard)
        targets_use = train_target[discard:]
        # X_aug = np.hstack([states_use, np.ones((states_use.shape[0],1))])

        # polynomial readout
        X_list = []
        for s in states_use:
            X_list.append(augment_state_with_squares(s))
        X_aug = np.array(X_list)  # shape => [T-discard, 2N+1]

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_

    def predict_autoregressive(self, initial_input, n_steps):
        preds = []
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            # x_aug = np.concatenate([self.x, [1.0]])
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
            current_in = out
        return np.array(preds)
    
    def predict_open_loop(self, test_input):
        preds = []
        for true_input in test_input:
            self._update(true_input)
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)


# CRJ

In [5]:
class CRJ3D:
    """
    Cycle Reservoir with Jumps (CRJ) for 3D->3D single-step tasks.
    We form a ring adjacency with an extra 'jump' edge in each row.
    This can help capture multiple timescales or delayed memory
    while retaining the easy ring structure.

    The adjacency is built as follows (reservoir_size = mod N):
      For each i in [0..N-1]:
        W[i, (i+1) % mod N] = 1.0
        W[i, (i+jump) % mod N] = 1.0
    Then we scale by 'spectral_radius.' We do an ESN update
    with readout [ x, x^2, 1 ] -> next step in R^3.
    """

    def __init__(self,
                 reservoir_size=300,
                 input_dim=62,
                 jump=10,                # offset for the jump
                 spectral_radius=0.95,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=42):
        """
        reservoir_size: how many nodes in the ring
        jump            : the offset for the 2nd connection from node i
        spectral_radius : scale adjacency
        input_scale     : scale factor for W_in
        leaking_rate    : ESN 'alpha'
        ridge_alpha     : ridge penalty for readout
        seed            : random seed
        """
        self.reservoir_size = reservoir_size
        self.jump = jump
        self.spectral_radius = spectral_radius
        self.input_scale = input_scale
        self.leaking_rate = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed = seed

        # build adjacency
        np.random.seed(self.seed)
        W = np.zeros((reservoir_size, reservoir_size))
        for i in range(reservoir_size):
            # cycle edge: i -> (i+1)%N
            W[i, (i+1) % reservoir_size] = 1.0
            # jump edge: i -> (i+jump)%N
            W[i, (i + self.jump) % reservoir_size] = 1.0

        # scale spectral radius
        W = scale_spectral_radius(W, self.spectral_radius)
        self.W = W

        # input weights => shape [N,3]
        np.random.seed(self.seed+100)
        W_in = (np.random.rand(reservoir_size, input_dim) - 0.5)*2.0*self.input_scale
        self.W_in = W_in

        # readout
        self.W_out = None
        self.x = np.zeros(self.reservoir_size)

    def reset_state(self):
        self.x = np.zeros(self.reservoir_size)

    def _update(self, u):
        """
        Single-step ESN update:
          x(t+1) = (1-alpha)*x(t) + alpha*tanh( W x(t) + W_in u(t) )
        """
        pre_activation = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre_activation)
        alpha = self.leaking_rate
        self.x = (1.0 - alpha)*self.x + alpha*x_new

    def collect_states(self, inputs, discard=100):
        """
        Teacher forcing => feed the real 3D inputs => gather states.
        Return (states_after_discard, states_discarded).
        """
        self.reset_state()
        states = []
        for val in inputs:
            self._update(val)
            states.append(self.x.copy())
        states = np.array(states)
        return states[discard:], states[:discard]

    def fit_readout(self, train_input, train_target, discard=100):
        """
        gather states => polynomial readout => solve ridge
        """
        states_use, _ = self.collect_states(train_input, discard=discard)
        target_use = train_target[discard:]
        X_list = []
        for s in states_use:
            # polynomial expansion => [ x, x^2, 1 ]
            X_list.append(augment_state_with_squares(s))
        X_aug = np.array(X_list)

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, target_use)
        self.W_out = reg.coef_  # shape => (3, 2N+1)

    def predict_autoregressive(self, initial_input, n_steps):
        """
        fully autoregressive => feed last output => next input
        """
        preds = []
        #self.reset_state()
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            big_x = augment_state_with_squares(self.x)
            out = self.W_out @ big_x  # shape => (3,)
            preds.append(out)
            current_in = out
        return np.array(preds)
        
    def predict_open_loop(self, test_input):
        preds = []
        for true_input in test_input:
            self._update(true_input)
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)


# MCI

In [6]:
class MCI3D:
    """
    Minimum Complexity Interaction ESN (MCI-ESN).

    This class implements the approach described in:
      "A Minimum Complexity Interaction Echo State Network"
        by Jianming Liu, Xu Xu, Eric Li (2024).
    
    The model structure:
      - We maintain two 'simple cycle' reservoirs (each of size N).
      - Each reservoir is a ring with weight = l, i.e. 
            W_res[i, (i+1)%N] = l
        plus the corner wrap from (N-1)->0, also = l. ##(unnecessary as already called for in the prev. line)
      - The two reservoirs interact via a minimal connection matrix: 
         exactly 2 cross-connections with weight = g. 
         (One might connect x2[-1], x2[-2], ... 
          But we do where reservoir1 sees x2[-1] 
          in one location, and reservoir2 sees x1[-1] likewise.)
      - Activation function in reservoir1 is cos(·), and in reservoir2 is sin(·).
      - They each have a separate input weight matrix: Win1 and Win2. 
        The final state is a linear combination 
           x(t) = h*x1(t) + (1-h)*x2(t).
      - Then we do a polynomial readout [x, x^2, 1] -> output.
      - We feed teacher forcing in collect_states, 
        then solve readout with Ridge regression.

    References:
      - Liu, J., Xu, X., & Li, E. (2024). 
        "A minimum complexity interaction echo state network," 
         Neural Computing and Applications.
    
    notes:
      - The reservoir_size is N for each reservoir, 
        so total param dimension is 2*N for states, 
        but we produce a single final "combined" state x(t) in R^N for readout.
      - The activation f1=cos(...) for reservoir1, f2=sin(...) for reservoir2, 
        as recommended by the paper for MCI-ESN.

    """

    def __init__(
        self,
        reservoir_size=250,
        cycle_weight=0.9,      # 'l' in the paper
        connect_weight=0.9,    # 'g' in the paper
        input_scale=0.2,
        leaking_rate=1.0,
        ridge_alpha=1e-6,
        combine_factor=0.1,    # 'h' in the paper
        seed=47,
        v1=0.6, v2=0.6         # fixed values for v1, v2
    ):
        """
        reservoir_size: N, size of each cycle reservoir 
        cycle_weight : l, ring adjacency weight in [0,1), ensures cycle synergy
        connect_weight: g, cross-connection weight between the two cycle reservoirs
        input_scale   : scale factor for input->reservoir weights
        leaking_rate  : ESN update alpha 
        ridge_alpha   : readout ridge penalty
        combine_factor: h in [0,1], to form x(t)= h*x1(t)+(1-h)*x2(t) as final combined state
        seed          : random seed
        """
        self.reservoir_size = reservoir_size
        self.cycle_weight   = cycle_weight
        self.connect_weight = connect_weight
        self.input_scale    = input_scale
        self.leaking_rate   = leaking_rate
        self.ridge_alpha    = ridge_alpha
        self.combine_factor = combine_factor
        self.seed           = seed
        self.v1 = v1
        self.v2 = v2

        # We'll define (and build) adjacency for each cycle, 
        # plus cross-connection for two sub-reservoirs.
        # We'll define 2 input weight mats: Win1, Win2.
        # We'll define states x1(t), x2(t).
        # We'll define readout W_out after training.

        self._build_mci_esn()

    def _build_mci_esn(self):
        """
        Build all the internal parameters: 
         - ring adjacency for each reservoir
         - cross-reservoir connection
         - input weights for each reservoir
         - initial states
        """
        np.random.seed(self.seed)

        N = self.reservoir_size

        # Build ring adjacency W_res in shape [N, N], with cycle_weight on ring
        W_res = np.zeros((N, N))
        for i in range(N):
            j = (i+1) % N
            W_res[j, i] = self.cycle_weight
        self.W_res = W_res  # shared by both sub-reservoirs

        # Build cross-connection W_cn for shape [N,N], 
        # minimal 2 nonzero elements. 
        # For the simplest approach from the paper:
        #   W_cn[0, N-1] = g, W_cn[1, N-2] = g or similar.
        # The paper's eq(7) suggests the last 2 elements in x(t) cross to first 2 in the other reservoir:
        # We'll do the simplest reference: if i=0 or i=1, we connect from the other reservoir's last or second-last. 
        # We'll define a function for each sub-res to pick up from the other sub-res. 
        # We can store them in separate arrays, or define them in code. 
        # We'll just store "We want index 0 to see x2[-1], index 1 to see x2[-2]."

        # But as done in the original code snippet from the paper:
        #   Wcn has
        # effectively 2 nonzero positions. We'll define that pattern:
        W_cn = np.zeros((N, N))
        # e.g. W_cn[0, N-1] = g, W_cn[N-1, N-2] = g or something. 
        # The paper example used W_cn = diag(0,g,...) plus the corner. We'll do the simplest:
        # let W_cn[0, N-1]=g, W_cn[1, N-2]=g.
        # This matches the minimal cross. 
        # For clarity we do:
        W_cn[0, N-1] = self.connect_weight
        if N>1:
            # W_cn[1, N-2] = self.connect_weight
            W_cn[N-1, 0] = self.connect_weight
        self.W_cn = W_cn

        # We'll define input weights for each sub-reservoir, shape [N, dim_input].
        # The paper sets them as eq(10) in the snippet, with different signs. 
        # We'll define them as parted. 
        # We define V1, V2 => shape [N, dim_input], with constant magnitude t1, t2, random sign. 
        # We'll do random. Need to check this in the paper again
        # We'll keep "two" separate. user can define input_scale but not two separate. 
        # We'll do the simplest approach: the absolute value is the same => input_scale, 
        # sign is random. Then we define Win1 = V1 - V2, Win2 = V1 + V2.
        # This is consistent with eq(10) from the paper.

        self.Win1 = None
        self.Win2 = None

        # We'll define states x1(t), x2(t). We'll do them after dimension known. 
        self.x1 = None
        self.x2 = None

        self.W_out = None

    def _init_substates(self):
        """
        Once we know reservoir_size, we define x1, x2 as zeros. 
        We'll call this in reset_state or at fit time.
        """
        N = self.reservoir_size
        self.x1 = np.zeros(N)
        self.x2 = np.zeros(N)

    def reset_state(self):
        if self.x1 is not None:
            self.x1[:] = 0.0
        if self.x2 is not None:
            self.x2[:] = 0.0

    def _update(self, u):
        """
        Single-step reservoir update.
        x1(t+1) = cos( Win1*u(t+1) + W_res*x1(t) + W_cn*x2(t) )
        x2(t+1) = sin( Win2*u(t+1) + W_res*x2(t) + W_cn*x1(t) )
        Then x(t)= h*x1(t+1) + (1-h)* x2(t+1).
        We'll define the leaky integration. 
        But the paper uses an approach with no leak? Be careful.
        We'll do the approach: x1(t+1)= (1-alpha)* x1(t) + alpha*cos(...).
        """
        alpha = self.leaking_rate

        # pre activation for reservoir1
        pre1 = self.Win1 @ u + self.W_res @ self.x1 + self.W_cn @ self.x2
        # reservoir1 uses cos
        new_x1 = np.cos(pre1)

        # reservoir2 uses sin
        pre2 = self.Win2 @ u + self.W_res @ self.x2 + self.W_cn @ self.x1
        new_x2 = np.sin(pre2)

        self.x1 = (1.0 - alpha)*self.x1 + alpha*new_x1
        self.x2 = (1.0 - alpha)*self.x2 + alpha*new_x2

    def _combine_state(self):
        """
        Combine x1(t), x2(t) => x(t) = h*x1 + (1-h)*x2
        """
        h = self.combine_factor
        return h*self.x1 + (1.0 - h)*self.x2

    def collect_states(self, inputs, discard=100):
        # We reset the reservoir to zero
        self.reset_state()
        states = []
        for t in range(len(inputs)):
            self._update(inputs[t])   # feed the REAL input from the dataset
            combined = self._combine_state()
            states.append(combined.copy())
        states = np.array(states)  # shape => [T, N]
        return states[discard:], states[:discard]


    def fit_readout(self, train_input, train_target, discard=100):
        """
        Build input weights if needed, gather states on the training data (teacher forcing),
        then solve a polynomial readout [x, x^2, 1]->train_target(t).

        train_input : shape [T, d_in]
        train_target: shape [T, d_out]
        discard     : # of states to discard for warmup
        """
        T = len(train_input)
        if T<2:
            raise ValueError("Not enough training data")

        d_in = train_input.shape[1]
        # d_out = train_target.shape[1]

        # built Win1, Win2
        if self.Win1 is None or self.Win2 is None:
            np.random.seed(self.seed+100)
            # build V1, V2 in shape [N, d_in]
            N = self.reservoir_size
            # V1 = (np.random.rand(N, d_in)-0.5)*2.0*self.input_scale
            # V2 = (np.random.rand(N, d_in)-0.5)*2.0*self.input_scale

            sign_V1 = np.random.choice([-1, 1], size=(N, d_in))
            sign_V2 = np.random.choice([-1, 1], size=(N, d_in))

            v1, v2 = self.v1, self.v2  # fixed values for V1, V2

            V1 = v1 * sign_V1 * self.input_scale
            V2 = v2 * sign_V2 * self.input_scale

            # eq(10): Win1= V1 - V2, Win2= V1 + V2
            self.Win1 = V1 - V2
            self.Win2 = V1 + V2

        # define x1, x2
        self._init_substates()

        # gather states
        states_use, _ = self.collect_states(train_input, discard=discard)
        target_use = train_target[discard:]  # shape => [T-discard, d_out]

        # polynomial readout
        X_list = []
        for s in states_use:
            X_list.append(augment_state_with_squares(s))
        X_aug = np.array(X_list)  # shape => [T-discard, 2N+1]

        # Solve ridge
        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, target_use)
        # W_out => shape [d_out, 2N+1]
        self.W_out = reg.coef_

    def predict_autoregressive(self, initial_input, n_steps):
        """
        Fully autoregressive: 
          We do not use teacher forcing, 
          we feed the model's last output as the next input 
        Typically, for MCI-ESN the paper does input(t+1) in R^d. 
        We do the test_input
        For multi-step chaotic forecast, we feed the model's output as input? 
        That means the system dimension d_in must match d_out. 
        """
        preds = []
        # re-init states
        #self._init_substates()

        # we assume initial_input => shape (d_in,)
        current_in = np.array(initial_input)

        for _ in range(n_steps):
            self._update(current_in)
            # read out
            combined = self._combine_state()
            big_x = augment_state_with_squares(combined)
            out = self.W_out @ big_x  # shape => (d_out,)

            preds.append(out)
            current_in = out  # feed output back as next input

        return np.array(preds)
        
    def predict_open_loop(self, test_input):
        preds = []
        for true_input in test_input:
            self._update(true_input)
            combined = self._combine_state()
            x_aug = augment_state_with_squares(combined)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)


# DeepESN

In [7]:
class DeepESN3D:
    """
    Deep Echo State Network (DeepESN) for multi-layered reservoir computing.
    Each layer has its own reservoir, and the states are propagated through layers.
    """

    def __init__(self,
                 num_layers=3,
                 reservoir_size=100,
                 input_dim=62,
                 spectral_radius=0.95,
                 connectivity=0.1,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 activation_choices=('tanh','relu','sin','linear'),
                 seed=42):
        """
        Parameters:
        - num_layers: Number of reservoir layers.
        - reservoir_size: Number of neurons in each reservoir layer.
        """
        self.num_layers = num_layers
        self.reservoir_size = reservoir_size
        self.spectral_radius = spectral_radius
        self.connectivity = connectivity
        self.input_scale = input_scale
        self.leaking_rate = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.activation_choices = activation_choices
        self.seed = seed

        # Initialize reservoirs and input weights for each layer
        self.reservoirs = []
        self.input_weights = []
        self.states = []

        np.random.seed(self.seed)
        for layer in range(num_layers):
            np.random.seed(seed + layer)
            W = np.random.randn(reservoir_size, reservoir_size) * 0.1
            mask = (np.random.rand(reservoir_size, reservoir_size) < self.connectivity)
            W = W * mask
            W = scale_spectral_radius(W, spectral_radius)
            self.reservoirs.append(W)

            if layer == 0 : 
                W_in = (np.random.rand(reservoir_size, input_dim) - 0.5) * 2.0 * input_scale
            else:
                W_in = (np.random.rand(reservoir_size, reservoir_size) - 0.5) * 2.0 * input_scale
            self.input_weights.append(W_in)

        np.random.seed(self.seed + 200)
        self.node_activations = np.random.choice(self.activation_choices, size=self.reservoir_size)
        
        self.W_out = None
        self.reset_state()

    def reset_state(self):
        """
        Reset the states of all reservoir layers.
        """
        self.states = [np.zeros(self.reservoir_size) for _ in range(self.num_layers)]

    def _apply_activation(self, act_type, val):
        return np.tanh(val)
        # if act_type=='tanh':
        #     return np.tanh(val)
        # elif act_type=='relu':
        #     return max(0.0, val)
        # elif act_type=='sin':
        #     return np.sin(val)
        # elif act_type=='linear':
        #     return val
        # else:
        #     return np.tanh(val)

    def _update_layer(self, layer_idx, u):
        """
        Update a single reservoir layer.
        """
        pre_activation = self.reservoirs[layer_idx] @ self.states[layer_idx]
        if layer_idx == 0:
            pre_activation += self.input_weights[layer_idx] @ u
        else:
            pre_activation += self.input_weights[layer_idx] @ self.states[layer_idx - 1]

        x_new = np.zeros_like(pre_activation)
        for i in range(self.reservoir_size):
            activation = self.node_activations[i]
            x_new[i] = self._apply_activation(activation, pre_activation[i])
        alpha = self.leaking_rate
        self.states[layer_idx] = (1.0 - alpha) * self.states[layer_idx] + alpha * x_new

    def collect_states(self, inputs, discard=100):
        self.reset_state()
        all_states = []
        for u in inputs:
            for layer_idx in range(self.num_layers):
                self._update_layer(layer_idx, u)
            all_states.append(np.concatenate(self.states))
        all_states = np.array(all_states)
        return all_states[discard:], all_states[:discard]

    def fit_readout(self, train_input, train_target, discard=100):
        """
        Train the readout layer using ridge regression.
        """
        states_use, _ = self.collect_states(train_input, discard=discard)
        targets_use = train_target[discard:]

        # Augment states with bias
        # X_aug = np.hstack([states_use, np.ones((states_use.shape[0], 1))])  # shape [T-discard, N*L+1]

        # Quadratic readout
        # Build augmented matrix [ x, x^2, 1 ]
        X_list = []
        for s in states_use:
            X_list.append( np.concatenate([s, s**2, [1.0]]) )
        X_aug = np.array(X_list)                                    # shape [T-discard, 2N*L+1]

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_

    def predict_open_loop(self, inputs):
        """
        Single-step-ahead inference on test data.
        """
        preds = []
        for u in inputs:
            for layer_idx in range(self.num_layers):
                self._update_layer(layer_idx, u)
            state = np.concatenate(self.states)
            # x_aug = np.concatenate([state, [1.0]])
            x_aug = np.concatenate([state, (state)**2, [1.0]])  # For quadrartic readout
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)

    def predict_autoregressive(self, initial_input, num_steps):
        """
        Autoregressive multi-step forecasting for num_steps
        """
        preds = []
        current_input = initial_input.copy()

        for _ in range(num_steps):
            for layer_idx in range(self.num_layers):
                self._update_layer(layer_idx, current_input)
            state = np.concatenate(self.states)
            # x_aug = np.concatenate([state, [1.0]])
            x_aug = np.concatenate([state, (state)**2, [1.0]])  # For quadrartic readout
            out = self.W_out @ x_aug
            preds.append(out)
            current_input = out
        
        return np.array(preds)

# HHLR [v1]

In [8]:


# ---------------------------------------------------------------------
# Utility helpers
# ---------------------------------------------------------------------
def scale_spectral_radius(W, desired_radius=0.95):
    """Affine-scale square matrix W so that its spectral radius equals desired_radius."""
    eigs = eigvals(W)
    current_radius = np.max(np.abs(eigs))
    if current_radius == 0:
        raise ValueError("Spectral radius of W is zero.")
    return W * (desired_radius / current_radius)

def augment_state_with_squares(x):
    """[x, x^2, 1]   (same convention as in CycleReservoir3D)."""
    return np.concatenate([x, x**2, [1.0]])

# ---------------------------------------------------------------------
# HH-LR class
# ---------------------------------------------------------------------
class HHLobeReservoir1:
    """
    Hemispherically-Hierarchical Lobe Reservoir (HH-LR).

    Topology:
      * 8 anatomical modules  —  L/R × {F, P, T, O}
      * Intra-module: Watts–Strogatz small-world graphs (ring+rewire)
      * Intra-hemisphere inter-lobe: distance-modulated shortcuts
      * Inter-hemisphere (callosal): sparse homotopic bridges
    """

    # --- anatomical bookkeeping -------------------------------------------------
    _LOBES   = ['F', 'P', 'T', 'O']
    _HEMIS   = ['L', 'R']
    # rough 2-D centroids (arbitrary units) for distance computation
    _CENTROIDS = {
        ('L', 'F'): (-1.0,  1.0),
        ('L', 'P'): (-1.0,  0.0),
        ('L', 'T'): (-1.0, -1.0),
        ('L', 'O'): (-1.0, -2.0),
        ('R', 'F'): ( 1.0,  1.0),
        ('R', 'P'): ( 1.0,  0.0),
        ('R', 'T'): ( 1.0, -1.0),
        ('R', 'O'): ( 1.0, -2.0),
    }

    # ------------------------------------------------------------------
    def __init__(self,
                 reservoir_size=800,
                 input_dim=128,
                 spectral_radius=0.9,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 # small-world / shortcut hyper-parameters
                 k_ring=8,
                 p_rewire_frontal=0.30,
                 p_rewire_other=0.10,
                 P_lat=0.04,
                 sigma=5.0,
                 P_call=0.01,
                 seed=42):
        """
        Parameters
        ----------
        reservoir_size   : total neuron count (split equally across the 8 modules if not divisible)
        input_dim        : dimensionality of input vector u_t  (128 for EEG band-power features)
        leaking_rate     : α in leaky-integrator update
        """
        self.N     = reservoir_size
        self.D_in  = input_dim
        self.rho   = spectral_radius
        self.in_scale   = input_scale
        self.alpha = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed  = seed

        # ------------------------------------------------------------------
        # 1) allocate neurons to the 8 modules as evenly as possible
        # ------------------------------------------------------------------
        base = self.N // 8
        counts = [base] * 8
        for i in range(self.N - base*8):
            counts[i] += 1
        self._module_slices = {}
        idx0 = 0
        for h in self._HEMIS:
            for l in self._LOBES:
                n = counts[len(self._module_slices)]
                self._module_slices[(h, l)] = slice(idx0, idx0 + n)
                idx0 += n

        # ------------------------------------------------------------------
        # 2) build adjacency matrix W according to HH-LR rules
        # ------------------------------------------------------------------
        rng = np.random.default_rng(self.seed)
        W = np.zeros((self.N, self.N), dtype=float)

        # 2.1 intra-module small-world wiring
        for (h, l), sl in self._module_slices.items():
            n_mod = sl.stop - sl.start
            k = min(k_ring, n_mod-1)  # guard tiny modules
            p_rewire = p_rewire_frontal if l == 'F' else p_rewire_other
            # ring lattice
            for i_local in range(n_mod):
                for m in range(1, k+1):
                    j_local = (i_local + m) % n_mod
                    i_glob = sl.start + i_local
                    j_glob = sl.start + j_local
                    W[i_glob, j_glob] = rng.standard_normal()
                    W[j_glob, i_glob] = rng.standard_normal()
            # rewire each existing edge with prob p_rewire
            for i_local in range(n_mod):
                for m in range(1, k+1):
                    if rng.random() < p_rewire:
                        # pick a random new target in same module (avoid self-loop)
                        j_local_new = rng.integers(0, n_mod-1)
                        if j_local_new >= i_local:
                            j_local_new += 1
                        i_glob = sl.start + i_local
                        j_glob_new = sl.start + j_local_new
                        # overwrite previous weight (both directions)
                        w_new = rng.standard_normal()
                        W[i_glob, j_glob_new] = w_new
                        W[j_glob_new, i_glob] = rng.standard_normal()

        # 2.2 intra-hemisphere inter-lobe shortcuts (distance-weighted)
        for h in self._HEMIS:
            for l1 in self._LOBES:
                for l2 in self._LOBES:
                    if l1 == l2:
                        continue
                    sl1 = self._module_slices[(h, l1)]
                    sl2 = self._module_slices[(h, l2)]
                    c1 = np.array(self._CENTROIDS[(h, l1)])
                    c2 = np.array(self._CENTROIDS[(h, l2)])
                    dist = np.linalg.norm(c1 - c2)
                    p_edge = P_lat * np.exp(-dist / sigma)
                    for i in range(sl1.start, sl1.stop):
                        mask = rng.random(sl2.stop - sl2.start) < p_edge
                        js = np.nonzero(mask)[0] + sl2.start
                        if js.size:
                            W[i, js] = rng.standard_normal(size=js.size)
                            W[js, i] = rng.standard_normal(size=js.size)

        # 2.3 inter-hemisphere callosal bridges (homotopic)
        for l in self._LOBES:
            sl_L = self._module_slices[('L', l)]
            sl_R = self._module_slices[('R', l)]
            for i in range(sl_L.start, sl_L.stop):
                mask = rng.random(sl_R.stop - sl_R.start) < P_call
                js = np.nonzero(mask)[0] + sl_R.start
                if js.size:
                    W[i, js] = rng.standard_normal(size=js.size)
                    W[js, i] = rng.standard_normal(size=js.size)

        # 2.4 spectral scaling
        W = scale_spectral_radius(W, self.rho)
        self.W = W

        # ------------------------------------------------------------------
        # 3) random input weights
        # ------------------------------------------------------------------
        rng = np.random.default_rng(self.seed + 1)
        self.W_in = (rng.random((self.N, self.D_in)) - 0.5) * 2.0 * self.in_scale

        # readout and state
        self.W_out = None
        self.x = np.zeros(self.N)

    # ------------------------------------------------------------------
    # ESN core methods (same signatures as CycleReservoir3D)
    # ------------------------------------------------------------------
    def reset_state(self):
        self.x = np.zeros(self.N)

    def _update(self, u):
        pre_activation = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre_activation)
        self.x = (1.0 - self.alpha) * self.x + self.alpha * x_new

    def collect_states(self, inputs, discard=100):
        """
        Parameters
        ----------
        inputs  : iterable / array of shape (T, input_dim)
        discard : number of initial time-steps to omit from training
        """
        self.reset_state()
        states = []
        for u in inputs:
            self._update(u)
            states.append(self.x.copy())
        return np.array(states[discard:]), np.array(states[:discard])

    def fit_readout(self, train_input, train_target, discard=100):
        """
        Ridge regression read-out; identical augmentation as baseline.
        """
        states_use, _ = self.collect_states(train_input, discard=discard)
        targets_use = train_target[discard:]
        X_aug = np.vstack([augment_state_with_squares(s) for s in states_use])

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_

    def predict_sequence(self, inputs):
        """
        Feed-forward prediction (no teacher forcing).  Suitable for classification:
        returns raw linear outputs; apply threshold/sigmoid externally.
        """
        preds = []
        self.reset_state()
        for u in inputs:
            self._update(u)
            preds.append(self.W_out @ augment_state_with_squares(self.x)) # quadratic lift-off
            #preds.append(self.W_out @ self.x)
        return np.array(preds)
    
    def predict_autoregressive(self, initial_input, n_steps):
        preds = []
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            out = self.W_out @ augment_state_with_squares(self.x)
            preds.append(out)
            current_in = out
        return np.array(preds)

# HHLR[v2]

In [9]:
# ---------------------------------------------------------------------
# Utility helpers
# ---------------------------------------------------------------------
def scale_spectral_radius(W, desired_radius=0.95):
    """Affine-scale W so that its spectral radius equals desired_radius."""
    eigs = eigvals(W)
    radius = np.max(np.abs(eigs))
    if radius == 0:
        raise ValueError("Spectral radius of W is zero.")
    return W * (desired_radius / radius)


def augment_state_with_squares(x):
    """Return [x, x², 1] (same convention as in CycleReservoir3D)."""
    return np.concatenate([x, x**2, [1.0]])


# ---------------------------------------------------------------------
# Hemispherically–Hierarchical Lobe Reservoir
# ---------------------------------------------------------------------
class HHLobeReservoir2:
    """
    Hemispherically-Hierarchical Lobe Reservoir (HH-LR)

    • 8 anatomical modules  —  L/R × {F, P, T, O}
    • Intra-module: Watts–Strogatz small-world graphs (ring + rewire)
    • Intra-hemisphere shortcuts: distance-modulated (exp decay)
    • Inter-hemisphere bridges: sparse homotopic callosal links
    • Lobe cardinalities follow MRI volume ratio 4 : 3 : 2 : 1
    """

    # bookkeeping ------------------------------------------------------
    _LOBES  = ['F', 'P', 'T', 'O']
    _HEMIS  = ['L', 'R']
    _CENTROIDS = {                             # rough 2-D montage coords
        ('L', 'F'): (-1.0,  1.0), ('L', 'P'): (-1.0,  0.0),
        ('L', 'T'): (-1.0, -1.0), ('L', 'O'): (-1.0, -2.0),
        ('R', 'F'): ( 1.0,  1.0), ('R', 'P'): ( 1.0,  0.0),
        ('R', 'T'): ( 1.0, -1.0), ('R', 'O'): ( 1.0, -2.0),
    }

    # ------------------------------------------------------------------
    def __init__(self,
                 reservoir_size=800,
                 input_dim=32,
                 spectral_radius=0.9,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 # small-world / shortcut hyper-parameters
                 k_ring=8,
                 p_rewire_frontal=0.30,
                 p_rewire_other=0.10,
                 P_lat=0.04,
                 sigma=5.0,
                 P_call=0.01,
                 seed=42):
        """
        reservoir_size : total number of neurons (N)
        input_dim      : dimension of u_t  (128 for EEG band-power features)
        leaking_rate   : α in leaky-integrator update
        """
        self.N          = reservoir_size
        self.D_in       = input_dim
        self.rho        = spectral_radius
        self.in_scale   = input_scale
        self.alpha      = leaking_rate
        self.ridge_alpha= ridge_alpha
        self.seed       = seed

        # --------------------------------------------------------------
        # 1) allocate neurons:  per-hemisphere 4 : 3 : 2 : 1 (F:P:T:O)
        # --------------------------------------------------------------
        ratio = {'F': 4, 'P': 3, 'T': 2, 'O': 1}
        total_w = sum(ratio.values())          # = 10
        half_N  = self.N // 2                  # neurons per hemisphere

        counts_per_hemi = {
            l: int(half_N * ratio[l] / total_w) for l in self._LOBES
        }

        # distribute rounding residuals (largest fractional remainder first)
        residual = half_N - sum(counts_per_hemi.values())
        if residual > 0:
            remainders = sorted(
                self._LOBES,
                key=lambda l: (half_N * ratio[l] / total_w) - counts_per_hemi[l],
                reverse=True
            )
            for l in remainders[:residual]:
                counts_per_hemi[l] += 1

        # build slice table
        self._module_slices = {}
        idx0 = 0
        for h in self._HEMIS:          # L then R
            for l in self._LOBES:      # F, P, T, O
                n = counts_per_hemi[l]
                self._module_slices[(h, l)] = slice(idx0, idx0 + n)
                idx0 += n
        assert idx0 == self.N, "Slice allocation error"

        # --------------------------------------------------------------
        # 2) construct adjacency matrix W according to HH-LR rules
        # --------------------------------------------------------------
        rng = np.random.default_rng(self.seed)
        W = np.zeros((self.N, self.N), dtype=float)

        # 2.1 intra-module small-world wiring
        for (h, l), sl in self._module_slices.items():
            n_mod = sl.stop - sl.start
            k = min(k_ring, n_mod - 1)
            p_rewire = p_rewire_frontal if l == 'F' else p_rewire_other

            # ring lattice
            for i_local in range(n_mod):
                for m in range(1, k + 1):
                    j_local = (i_local + m) % n_mod
                    i_glob  = sl.start + i_local
                    j_glob  = sl.start + j_local
                    W[i_glob, j_glob] = rng.standard_normal()
                    W[j_glob, i_glob] = rng.standard_normal()

            # random rewiring
            for i_local in range(n_mod):
                for m in range(1, k + 1):
                    if rng.random() < p_rewire:
                        j_local_new = rng.integers(0, n_mod - 1)
                        if j_local_new >= i_local:
                            j_local_new += 1
                        i_glob = sl.start + i_local
                        j_glob_new = sl.start + j_local_new
                        w_new = rng.standard_normal()
                        W[i_glob, j_glob_new] = w_new
                        W[j_glob_new, i_glob] = rng.standard_normal()

        # 2.2 intra-hemisphere distance-weighted shortcuts
        for h in self._HEMIS:
            for l1 in self._LOBES:
                for l2 in self._LOBES:
                    if l1 == l2:
                        continue
                    sl1 = self._module_slices[(h, l1)]
                    sl2 = self._module_slices[(h, l2)]
                    d = np.linalg.norm(
                        np.array(self._CENTROIDS[(h, l1)]) -
                        np.array(self._CENTROIDS[(h, l2)])
                    )
                    p_edge = P_lat * np.exp(-d / sigma)
                    for i in range(sl1.start, sl1.stop):
                        mask = rng.random(sl2.stop - sl2.start) < p_edge
                        js = np.nonzero(mask)[0] + sl2.start
                        if js.size:
                            W[i, js] = rng.standard_normal(size=js.size)
                            W[js, i] = rng.standard_normal(size=js.size)

        # 2.3 inter-hemisphere homotopic callosal links
        for l in self._LOBES:
            sl_L = self._module_slices[('L', l)]
            sl_R = self._module_slices[('R', l)]
            for i in range(sl_L.start, sl_L.stop):
                mask = rng.random(sl_R.stop - sl_R.start) < P_call
                js = np.nonzero(mask)[0] + sl_R.start
                if js.size:
                    W[i, js] = rng.standard_normal(size=js.size)
                    W[js, i] = rng.standard_normal(size=js.size)

        # 2.4 spectral scaling to enforce ESP
        self.W = scale_spectral_radius(W, self.rho)

        # --------------------------------------------------------------
        # 3) random input weights
        # --------------------------------------------------------------
        rng = np.random.default_rng(self.seed + 1)
        self.W_in = (rng.random((self.N, self.D_in)) - 0.5) * 2.0 * self.in_scale

        # initialize state & read-out
        self.x = np.zeros(self.N)
        self.W_out = None

    # ------------------------------------------------------------------
    # ESN core methods --------------------------------------------------
    # ------------------------------------------------------------------
    def reset_state(self):
        self.x.fill(0.0)

    def _update(self, u):
        pre = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre)
        self.x = (1.0 - self.alpha) * self.x + self.alpha * x_new

    def collect_states(self, inputs, discard=100):
        self.reset_state()
        states = []
        for u in inputs:
            self._update(u)
            states.append(self.x.copy())
        return np.array(states[discard:]), np.array(states[:discard])

    # ------------------------------------------------------------------
    # read-out ----------------------------------------------------------
    # ------------------------------------------------------------------
    def fit_readout(self, train_input, train_target, discard=100):
        states_use, _ = self.collect_states(train_input, discard)
        targets_use = train_target[discard:]
        X_aug = np.vstack([augment_state_with_squares(s) for s in states_use])

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_

    def predict_sequence(self, inputs):
        preds = []
        self.reset_state()
        for u in inputs:
            self._update(u)
            preds.append(self.W_out @ augment_state_with_squares(self.x))
        return np.array(preds)
    
    def predict_autoregressive(self, initial_input, n_steps):
        preds = []
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            out = self.W_out @ augment_state_with_squares(self.x)
            preds.append(out)
            current_in = out
        return np.array(preds)


### NRMSE

In [10]:
def evaluate_nrmse(all_preds, test_target, horizons):
    """
    Evaluate model performance over multiple prediction horizons
    for teacher-forced single-step forecasting or autoregressive rollout.
    """
    horizon_nrmse = {}
    for horizon in horizons:
        preds = all_preds[:horizon]
        targets = test_target[:horizon]
        squared_errors = (preds - targets) ** 2
        variance = np.var(targets, axis=0)
        variance[variance == 0] = 1e-8  # avoid divide-by-zero
        nrmse = np.sqrt(np.sum(squared_errors) / (horizon * np.sum(variance)))
        horizon_nrmse[horizon] = nrmse
    return horizon_nrmse

### VPT

In [11]:
def compute_valid_prediction_time(y_true, y_pred, test_time, lyapunov_time, threshold=0.4):
    y_mean = np.mean(y_true, axis=0)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))

    error = y_true - y_pred
    squared_error = np.sum(error**2, axis=1)
    delta = squared_error / denom

    idx_exceed = np.where(delta > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = test_time[-1]
    else:
        T_VPT = test_time[idx_exceed[0]]

    ratio = T_VPT / lyapunov_time

    return T_VPT, ratio

### ADev

In [12]:
def compute_attractor_deviation(predictions, targets, cube_size=(0.1, 0.1, 0.1)):
    """
    Compute the Attractor Deviation (ADev) metric.

    Parameters:
        predictions (numpy.ndarray): Predicted trajectories of shape (n, 3).
        targets (numpy.ndarray): True trajectories of shape (n, 3).
        cube_size (tuple): Dimensions of the cube (dx, dy, dz).

    Returns:
        float: The ADev metric.
    """
    # Define the cube grid based on the range of the data and cube size
    min_coords = np.min(np.vstack((predictions, targets)), axis=0)
    max_coords = np.max(np.vstack((predictions, targets)), axis=0)

    # Create a grid of cubes
    grid_shape = ((max_coords - min_coords) / cube_size).astype(int) + 1

    # Initialize the cube occupancy arrays
    pred_cubes = np.zeros(grid_shape, dtype=int)
    target_cubes = np.zeros(grid_shape, dtype=int)

    # Map trajectories to cubes
    pred_indices = ((predictions - min_coords) / cube_size).astype(int)
    target_indices = ((targets - min_coords) / cube_size).astype(int)

    # Mark cubes visited by predictions and targets
    for idx in pred_indices:
        pred_cubes[tuple(idx)] = 1
    for idx in target_indices:
        target_cubes[tuple(idx)] = 1

    # Compute the ADev metric
    adev = np.sum(np.abs(pred_cubes - target_cubes))
    return adev

### PSD

In [13]:
def compute_psd(y, dt=0.01):
    z = y[:, 2]  # Extract Z-component
    
    # Compute PSD using Welch’s method
    freqs, psd = welch(z, fs=1/dt, window='hamming', nperseg=len(z))  # Using Hamming window
    
    return freqs, psd

# BCI

In [14]:
def create_delay_embedding(signal, embed_dim):
    L = len(signal) - embed_dim + 1
    emb = np.zeros((L, embed_dim))
    for i in range(L):
        emb[i, :] = signal[i:i+embed_dim]
    return emb

In [15]:
def create_delay_embedding(signal, embed_dim):
    L = len(signal) - embed_dim + 1
    emb = np.zeros((L, embed_dim))
    for i in range(L):
        emb[i, :] = signal[i:i+embed_dim]
    return emb

# ─── Load Data ─────────────────────────────────────────────────────────────
data_dir = 'datasets/BCI_Competion4_dataset4'
comp_path = os.path.join(data_dir, 'sub1_comp.mat')

comp_data = sio.loadmat(comp_path)
X_all = comp_data['train_data']       # shape: (timepoints × channels)
Y_all = comp_data['train_dg']         # shape: (timepoints × 5)

# ─── Parameters ────────────────────────────────────────────────────────────
N_train = 15000
N_test = 10000
emb_dim = 3

# ─── Select and Normalize First Channel ────────────────────────────────────
u = X_all[:, 0]
u_norm = (u - np.min(u)) / (np.max(u) - np.min(u))

v = Y_all[:, 0]
v_norm = (v - np.min(v)) / (np.max(v) - np.min(v))

# ─── Delay Embed Input Signal ──────────────────────────────────────────────
inputs = create_delay_embedding(u_norm, emb_dim)

# ─── Delay Embed First Finger Flexion ──────────────────────────────────────
targets = create_delay_embedding(v, emb_dim)

# ─── Split Train/Test ──────────────────────────────────────────────────────
train_input = inputs[:N_train]
train_target = targets[:N_train]
test_input = inputs[N_train+1:N_train+1+N_test]
test_target = targets[N_train+1:N_train+1+N_test]

# ─── Summary ───────────────────────────────────────────────────────────────
print(f"Train input shape:  {train_input.shape}")
print(f"Train target shape: {train_target.shape}")
print(f"Test input shape:   {test_input.shape}")
print(f"Test target shape:  {test_target.shape}")

Train input shape:  (15000, 3)
Train target shape: (15000, 3)
Test input shape:   (10000, 3)
Test target shape:  (10000, 3)


In [ ]:
horizons = [300, 600, 1000]

nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:
    hhlr = HHLobeReservoir1(
        reservoir_size=500,
        input_dim=emb_dim,
        spectral_radius=0.99,
        input_scale=0.001,
        leaking_rate=0.3,
        ridge_alpha=1e-07,
        k_ring=30,
        p_rewire_frontal=0.60,
        p_rewire_other=0.10,
        P_lat=0.04,
        sigma=5.0,
        P_call=0.01,
        seed=seed
    )
    hhlr.fit_readout(train_input, train_target, discard=5000)
    hhlr_preds = hhlr.predict_sequence(test_input)
    hhlr_nrmse = evaluate_nrmse(hhlr_preds, test_target, horizons)
    nrmse_dict['HHLR[v1]'].append(hhlr_nrmse)

In [147]:
model_names = ['HHLR[v1]']

print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
header = "Horizon".ljust(10) + "".join([model.ljust(18) for model in model_names])
print(header)
print("-" * 140)

for horizon in horizons:
    row = f"{str(horizon):<10}"
    for model in model_names:
        model_vals = [np.mean(run[horizon]) for run in nrmse_dict[model]]
        mean = np.mean(model_vals)
        std = np.std(model_vals)
        row += f"{mean:.4f} ± {std:.4f}".ljust(18)
    print(row)


NRMSE for Different Prediction Horizons:
--------------------------------------------------------------------------------------------------------------------------------------------
Horizon   HHLR[v1]          
--------------------------------------------------------------------------------------------------------------------------------------------
300       1.2056 ± 0.0313   
600       1.2795 ± 0.0220   
1000      1.4193 ± 0.0097   


In [ ]:
for seed in seeds:
    esn = ESN3D(
        reservoir_size=500,
        input_dim=62,
        spectral_radius=0.95,
        connectivity=0.05,
        input_scale=0.2,
        leaking_rate=0.8,
        ridge_alpha=1e-6,
        seed=seed
    )
    esn.fit_readout(train_input, train_target, discard=5000)
    esn_preds = esn.predict(test_target)
    esn_nrmse = evaluate_nrmse(esn_preds, Y_test, all_horizons)
    nrmse_dict['ESN'].append(esn_nrmse)

KeyboardInterrupt: 

In [ ]:
# for seed in seeds:
#     cycle_res = CycleReservoir3D(
#         reservoir_size=500,
#         input_dim=62,
#         cycle_weight = 0.8,
#         spectral_radius=0.95,
#         input_scale=0.2,
#         leaking_rate=0.8,
#         ridge_alpha=1e-6,
#         seed=seed
#     )
#     cycle_res.fit_readout(X_train, Y_train, discard=5000)
#     cycle_res_preds = cycle_res.predict(X_test)
#     cycle_res_nrmse = evaluate_nrmse(cycle_res_preds, Y_test, all_horizons)
#     nrmse_dict['SCR'].append(cycle_res_nrmse)


# for seed in seeds:
#     crj = CRJRes3D(
#         reservoir_size=500,
#         input_dim=62,
#         edge_weight=0.8,
#         jump=15,
#         spectral_radius=0.95,
#         input_scale=0.2,
#         leaking_rate=0.8,
#         ridge_alpha=1e-6,
#         seed=seed
#     )
#     crj.fit_readout(X_train, Y_train, discard=5000)
#     crj_preds = crj.predict(X_test)
#     crj_nrmse = evaluate_nrmse(crj_preds, Y_test, all_horizons)
#     nrmse_dict['CRJ'].append(crj_nrmse)


# for seed in seeds:
#     mci_esn = MCIESN3D(
#         reservoir_size=250,
#         input_dim=62,
#         cycle_weight=0.8,
#         connect_weight=0.8,
#         combine_factor=0.1,
#         v1=0.03,
#         v2=0.03,
#         spectral_radius=0.95,
#         leaking_rate=0.8,
#         ridge_alpha=1e-6,
#         seed=seed
#     )
#     mci_esn.fit_readout(X_train, Y_train, discard=5000)
#     mci_esn_preds = mci_esn.predict(X_test)
#     mci_esn_nrmse = evaluate_nrmse(mci_esn_preds, Y_test, all_horizons)
#     nrmse_dict['MCI-ESN'].append(mci_esn_nrmse)


# for seed in seeds:
#     deepesn = DeepESN3D(
#         num_layers=5,
#         reservoir_size=100,
#         input_dim=62,
#         spectral_radius=0.95,
#         input_scale=0.2,
#         leaking_rate=0.8,
#         ridge_alpha=1e-6,
#         seed=seed
#     )
#     deepesn.fit_readout(X_train, Y_train, discard=5000)
#     deepesn_preds = deepesn.predict(X_test)
#     deepesn_nrmse = evaluate_nrmse(deepesn_preds, Y_test, all_horizons)
#     nrmse_dict['DeepESN'].append(deepesn_nrmse)

In [96]:
# import pickle

# with open('BCI Baseline results/nrmse_dict_BCI2.pkl', 'wb') as f:
#     pickle.dump(nrmse_dict, f)